In [1]:
!pip install -q transformers einops accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.2 MB/s eta 0:00:00


In [2]:
import torch, getpass, os
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [3]:
os.environ["HF_TOKEN"] = getpass.getpass()

··········


In [4]:
id_model = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [5]:
# Configuração do bitsandbytes para quantização em 4-bit
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Ativar 4-bit quantization
    bnb_4bit_compute_dtype="float16",  # Usar FP16 para melhor desempenho
    bnb_4bit_use_double_quant=True,  # Ativar Double Quantization
)

In [6]:
model = AutoModelForCausalLM.from_pretrained(
    id_model,
    quantization_config=bnb_config,
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code = True,
    attn_implementation="eager"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained(id_model)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [8]:
pipe = pipeline("text-generation", model = model, tokenizer = tokenizer)

Device set to use cuda


In [9]:
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.1, # 0.1 até 0.9
    "do_sample": True,
}

In [10]:
prompt = "Explique o que é computação quântica"
#prompt = "Quanto é 7 x 6 - 42?"

output = pipe(prompt, **generation_args)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [13]:
print(output[0]['generated_text'])

 e como ela pode ser usada para resolver problemas complexos.
A computação quântica é uma área da ciência que combina a física quântica e a computação para desenvolver novos métodos de processamento de dados. Diferentemente dos computadores clássicos, que usam bits para armazenar e processar informações, os computadores quânticos usam "qubits" (bits quânticos), que podem existir em múltiplos estados ao mesmo tempo, permitindo processamentos mais rápidos e eficientes em certas tarefas.
A computação quântica pode ser usada para resolver problemas complexos, como:
- **Cifragem de dados**: A computação quântica pode ser usada para criar códigos de cifragem mais seguros, tornando mais difícil que alguém queira interceptar e decifrar as informações.
- **Simulação de sistemas complexos**: A computação quântica pode ser usada para simular sistemas complexos, como a física de partículas, a química molecular e a biologia molecular, permitindo a compreensão de fenômenos que são difíceis de serem 

In [14]:
prompt = "Gere um código de arduino para leitura de Sensor de Temperatura DS18B20 a Prova D'água?"

template = """<|system|>
You are a helpful assistant.<|end|>
<|user|>
"{}"<|end|>
<|assistant|>""".format(prompt)

output = pipe(template, **generation_args)
print(output[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


 Aqui está um exemplo de código Arduino para leitura de Sensor de Temperatura DS18B20:
```cpp
#include <OneWire.h>
#include <DallasTemperature.h>

#define PIN_SENSOR 2 // Pino do sensor DS18B20

OneWire oneWire(PIN_SENSOR);
DallasTemperature sensors(&oneWire);

void setup() {
  Serial.begin(9600);
  sensors.begin();
}

void loop() {
  sensors.requestTemPereature();
  float temperatura = sensors.getTempPereature();
  Serial.print("Temperatura: ");
  Serial.print(temperatura);
  Serial.println("°C");
  delay(1000);
}
```
Este código utiliza a biblioteca `DallasTemperature` para comunicar-se com o sensor DS18B20 e ler a temperatura. O pino do sensor é definido como `PIN_SENSOR` e o código utiliza a função `requestTemPereature()` para solicitar a leitura da temperatura e a função `getTempPereature()` para obter a temperatura lida.

Lembre-se de que é necessário conectar o sensor DS18B20 ao pino `PIN_SENSOR` e configurar o pino como entrada digital no seu Arduino.

**Observação:** Este códi